In [5]:
from google.colab import drive
drive.mount('/content/drive')
import sys
import os
import numpy as np
import glob
import shutil
import matplotlib.pyplot as plt
import tarfile
import cv2
#import TensorFlow and Kera Layers
from skimage.transform import resize
from sklearn.model_selection import train_test_split

from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization
from tensorflow.python.keras.layers.convolutional import Conv2D, MaxPooling2D 
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.applications import VGG19, ResNet50, InceptionV3
from tensorflow.python.keras.applications.vgg19 import preprocess_input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#Task 1 download the flower dataset
#unzip the file 
flower_file = tarfile.open('drive/My Drive/flower/flower_photos.tgz', "r:gz")
flower_file.extractall('unzipped_folder')
flower_file.close()
base_path = 'unzipped_folder/flower_photos/'
categories = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

#load the file
fnames = []
for category in categories:
    flower_folder = os.path.join(base_path, category)
    file_names = os.listdir(flower_folder)
    full_path = [os.path.join(flower_folder, file_name) for file_name in file_names]
    fnames.append(full_path)
    
#load the images
images = []
for names in fnames:
    one_category_images = [cv2.imread(name) for name in names if (cv2.imread(name)) is not None]
    images.append(one_category_images)

#resize all the images 
img_width, img_height = 256, 256
resized_images = []
for i,imgs in enumerate(images):
    resized_images.append([cv2.resize(img, (img_width, img_height), interpolation = cv2.INTER_CUBIC) for img in imgs])
#Task 1 (split dataset to 80% of training and 20% of validation)
train_images = []
val_images = []
for imgs in resized_images:
    train, test = train_test_split(imgs, train_size=0.8, test_size=0.2)
    train_images.append(train)
    val_images.append(test)

#Create Labels
#Training datasets
len_train_images = [len(imgs) for imgs in train_images]
train_categories = np.zeros((np.sum(len_train_images)), dtype='uint8')
for i in range(5):
    if i is 0:
        train_categories[:len_train_images[i]] = i
    else:
        train_categories[np.sum(len_train_images[:i]):np.sum(len_train_images[:i+1])] = i
#validation datasets
len_val_images = [len(imgs) for imgs in val_images]   
val_categories = np.zeros((np.sum(len_val_images)), dtype='uint8')
for i in range(5):
    if i is 0:
        val_categories[:len_val_images[i]] = i
    else:
        val_categories[np.sum(len_val_images[:i]):np.sum(len_val_images[:i+1])] = i  
        
tmp_train_imgs = []
tmp_val_imgs = []
for imgs in train_images:
    tmp_train_imgs += imgs
for imgs in val_images:
    tmp_val_imgs += imgs
train_images = np.array(tmp_train_imgs)
val_images = np.array(tmp_val_imgs)

#Convert image data to numpy array    
train_data = train_images.astype('float32')
val_data = val_images.astype('float32')
train_images = np.array(tmp_train_imgs)
val_images = np.array(tmp_val_imgs)        
train_labels = np_utils.to_categorical(train_categories, len(categories))
val_labels = np_utils.to_categorical(val_categories, len(categories)) 

#Shuffle the dataset
seed = 100
np.random.seed(seed)
np.random.shuffle(train_data)
np.random.seed(seed)
np.random.shuffle(train_labels)
np.random.seed(seed)
np.random.shuffle(val_data)
np.random.seed(seed)
np.random.shuffle(val_labels)


In [0]:
#Task 2
#Build convolutional neural network model
def create_model():
    model = Sequential()
    model.add(Conv2D(32, (3,3), padding='same', input_shape=train_data.shape[1:], activation='relu', name='conv_1'))
    model.add(Conv2D(32, (3,3), activation='relu', name='conv_2'))
    model.add(MaxPooling2D(pool_size=(2,2), name='maxpool_1'))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3,3), padding='same', activation='relu', name='conv_3'))
    model.add(Conv2D(64, (3,3), activation='relu', name='conv_4'))
    model.add(MaxPooling2D(pool_size=(2,2), name='maxpool_2'))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(128, (3,3), padding='same', activation='relu', name='conv_5'))
    model.add(Conv2D(128, (3,3), activation='relu', name='conv_6'))
    model.add(MaxPooling2D(pool_size=(2,2), name='maxpool_3'))

    model.add(Flatten())
    model.add(Dense(512, activation='relu', name='dense_1'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu', name='dense_2'))
    model.add(Dense(len(categories), name='output'))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc']) # optimizer=RMSprop(lr=0.001)
    
    return model
model_scratch = create_model()

# Using generator
# Adding rescale, rotation_range, width_shift_range, height_shift_range,
# shear_range, zoom_range, and horizontal flip to our ImageDataGenerator
batch_size = 32
epochs1 = 50
epochs2 = 10
epochs3 = 30
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.4,
    height_shift_range=0.4,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True
)
val_datagen = ImageDataGenerator(
    rescale=1./255,)
train_generator = train_datagen.flow(
    train_data,
    train_labels,
    batch_size=batch_size
)
    
val_generator = val_datagen.flow(
    val_data,
    val_labels,
    batch_size=batch_size
)
#Using fit_generator
model_scratch_info = model_scratch.fit_generator(
    generator=train_generator,  
    epochs=epochs1, 
    validation_steps=len(val_data)/batch_size, 
    steps_per_epoch=len(train_data)/batch_size,   
    validation_data=val_generator, 
    verbose=2
)


Epoch 1/50
92/91 - 65s - loss: 1.5686 - acc: 0.2723 - val_loss: 1.4710 - val_acc: 0.3342
Epoch 2/50


In [0]:
#Task 3 using VGG
def create_model_from_VGG19():

    model = VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
   
    for layer in model.layers[:1]:
      layer.trainable = False
      
    x = model.output
    x = Flatten()(x)
    x = Dense(1024, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation="relu")(x)
    predictions = Dense(len(categories), activation="softmax")(x)
    
    final_model = Model(inputs = model.input, outputs = predictions)
    
    final_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc']) # optimizer=RMSprop(lr=0.001)
    
    return final_model
  
model_VGG19 = create_model_from_VGG19()

#Complie the model
model_VGG19_info = model_VGG19.fit_generator(
    generator=train_generator, 
    steps_per_epoch=len(train_data)/batch_size,   # -> 106 # images 3392 = steps * batch_size = 106 * 32 
    epochs=epochs2, 
    validation_steps=len(val_data)/batch_size, # -> 26 # images 832 = steps * batch_size = 26 * 32
    validation_data=val_generator,
    verbose=2
)


In [0]:
#Task4 Data augmention
image_size = 224

datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   horizontal_flip=True,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2)

datagen.fit(train_data)


vggmodel.fit_generator(datagen.flow(train_data, train_labels, batch_size=32),
                    steps_per_epoch=len(train_data) / 32, epochs=10)